In [239]:
import pandas as pd
import re
from pathlib import Path

## Unidades

In [240]:
unidades = pd.read_csv("../data/transformed/unidades.csv", delimiter=";")

unidades.head()

,nome,sigla,endereco,site,telefone,email,cnpj,uasg,autoridade_maxima_unidade
0,CÂMPUS ÁGUAS LINDAS,LIN,"Rua 21, Área Especial 4, Jardim Querência, CEP...",http://www.ifg.edu.br/aguaslindas,(61) 3618-9850,gabinete.aguaslindas@ifg.edu.br,10.870.883/0014-69,158640,Tiago Gomes de Araujo
1,CÂMPUS ANÁPOLIS,ANA,"Avenida Pedro Ludovico, s/ nº, Reny Cury, CEP ...",http://www.ifg.edu.br/anapolis,(62) 3703-3350,gabinete.anapolis@ifg.edu.br,10.870.883/0009-00,158522,Katia Cilene Costa Fernandes
2,CÂMPUS APARECIDA DE GOIÂNIA,APA,Avenida Universitária Vereador Vagner da Silva...,http://www.ifg.edu.br/aparecida,(62) 3507-5950,gabinete.aparecida@ifg.edu.br,10.870.883/0010-35,158610,Eduardo de Carvalho Rezende
3,CÂMPUS CIDADE DE GOIÁS,GOI,"Rua 02, Quadra 10, Lote 01 a 15, Residencial B...",http://www.ifg.edu.br/goias,(62) 3371-9000,gabinete.goias@ifg.edu.br,10.870.883/0011-16,158611,Sandro Ramos de Lima
4,CÂMPUS FORMOSA,FOR,"Rua 64 esquina com Rua 11, S/Nº, Expansão Parq...",http://www.ifg.edu.br/formosa,(61) 3642 9450,gabinete.formosa@ifg.edu.br,10.870.883/0007-30,158523,Thiago Goncalves Dias


## Coluna Endereço

- Atomizar os dados de endereço, dispersando-os em colunas

### Tratamento dos números

Os números do endereço podem envolver quadra e lote, ou quilômetro da rodovia, logo um simples split não é suficiente

In [241]:
unidades["endereco"][0]

'Rua 21, Área Especial 4, Jardim Querência, CEP 72910-733, ÁGUAS LINDAS DE GOIÁS - GO'

In [242]:
enderecos_raw = unidades.values[:, 2]
enderecos_cleaned = []

# Separa os campos em um array e faz o strip() em cada um
for endereco in endereco_raw:
    row = []
    
    for field in endereco.split(","):
        row.append(field.strip())
        
    enderecos_cleaned.append(row)

# Armazena os números antigos para comparar o resultado do pré-processamento
numeros_before = []

for endereco in enderecos_cleaned:
    temp = " ".join(endereco[1:3]) if re.match(r"(quadra |lote |qd\.? |lt\.? )", " ".join(endereco[1:3]), flags=re.IGNORECASE) else endereco[1]
    numeros_before.append(temp)
                    
# Padroniza a nomenclatura da numeração
# Quadra  -> Qd.
# Lote    -> Lt.
# nº 64   -> 64
# km 89   -> Km 89
numeros = []

for endereco in enderecos_cleaned:
    sem_numero_regex = r"s\/\s*nº"
    num_ordinal_regex = r"nº\s*"
    quilometro_regex = r"km\s*"
    
    if re.match(r"(qd\.?\s|lt\.?\s)", " ".join(endereco[1:3]).lower(), flags=re.IGNORECASE):
        numero_composto = " ".join(endereco[1:3])
        numero_composto = re.sub(r"qd\.?", "Qd.", numero_composto, flags=re.IGNORECASE)
        numero_composto = re.sub(r"lt\.?", "Lt.", numero_composto, flags=re.IGNORECASE)
        numeros.append(numero_composto)
    elif "quadra" in endereco[1].lower() and "lote" in endereco[2].lower():
        numeros.append(" ".join(endereco[1:3]).lower().replace("quadra", "Qd.").replace("lote", "Lt."))
    elif "quadra" in endereco[1].lower():
        numeros.append(endereco[1].lower().replace("quadra", "Qd."))
    elif re.match(sem_numero_regex, endereco[1], re.IGNORECASE) != None:
        numeros.append(re.sub(sem_numero_regex, "S/N", endereco[1], flags=re.IGNORECASE))
    elif re.match(num_ordinal_regex, endereco[1], re.IGNORECASE):
        numeros.append(re.sub(num_ordinal_regex, "", endereco[1], flags=re.IGNORECASE))
    elif re.match(quilometro_regex, endereco[1], re.IGNORECASE):
        numeros.append(re.sub(quilometro_regex, "Km ", endereco[1], flags=re.IGNORECASE))
    else:
        numeros.append(endereco[1])
        
pd.DataFrame({"antes": numeros_before, "depois": numeros})

,antes,depois
0,Área Especial 4,Área Especial 4
1,s/ nº,S/N
2,Qd. 1 Lt. 1-A,Qd. 1 Lt. 1-A
3,Quadra 10 Lote 01 a 15,Qd. 10 Lt. 01 a 15
4,S/Nº,S/N
5,nº 46,46
6,S/N,S/N
7,S/Nº,S/N
8,nº 55,55
9,nº 775,775


### Tratamento dos CEPs

Os CEPs dos endereços estão seguindo o padrão: "CEP XXXXX-XXX" mas para diminuir a redundância do grafo, queremos apenas o número do CEP

In [243]:
ceps = []
ceps_before = []

cep_regex = r"CEP\s*"

for endereco in enderecos_cleaned:
    # Armazenar os antigos CEPs para comparar os resultados do pré-processamento
    ceps_before.append(endereco[-2])
    
    cep = re.sub(cep_regex, "", endereco[-2], flags=re.IGNORECASE)
    
    if cep.find("-") == -1:
        cep = "-".join([cep[:5], cep[-3:]])
    
    ceps.append(cep)
    
pd.DataFrame({"antes": ceps_before, "depois": ceps})

,antes,depois
0,CEP 72910-733,72910-733
1,CEP 75131-457,75131-457
2,CEP 74968-755,74968-755
3,CEP 76600-000,76600-000
4,CEP 73813-816,73813-816
5,CEP 74055-110,74055-110
6,CEP 74393-290,74393-290
7,CEP 75402-556,75402-556
8,CEP 75524-010,75524-010
9,CEP 75804-714,75804-714


### Tratamento do nome da cidade

O nome das cidades estão com a UF no final, mas sabemos que os IFG do dataset estão todos localizados em Goiás, logo para os dados ficarem mais limpos é necessário remover a sigla da UF do nome da cidade.

In [244]:
cidades = []

for endereco in enderecos_cleaned:
    cidades.append(endereco[-1][0:-5].upper())
    
pd.DataFrame(cidades, columns=["cidade"])

,cidade
0,ÁGUAS LINDAS DE GOIÁS
1,ANÁPOLIS
2,APARECIDA DE GOIÂNIA
3,CIDADE DE GOIÁS
4,FORMOSA
5,GOIÂNIA
6,GOIÂNIA
7,INHUMAS
8,ITUMBIARA
9,JATAÍ


### Extração do logradouro e bairro do dataset inicial

In [245]:
logradouros = []
bairros = []

for endereco in enderecos_cleaned:
    logradouros.append(endereco[0])
    bairros.append(endereco[-3])
    

pd.DataFrame({"logradouro": logradouros, "bairro": bairros})

,logradouro,bairro
0,Rua 21,Jardim Querência
1,Avenida Pedro Ludovico,Reny Cury
2,Avenida Universitária Vereador Vagner da Silva...,Parque Itatiaia
3,Rua 02,Centro
4,Rua 64 esquina com Rua 11,Expansão Parque Lago
5,Rua 75,Centro
6,Rua FP. 31,Recreio dos Funcionários Públicos
7,Av. Universitária,Vale das Goiabeiras
8,Avenida Furnas,Bairro Village Imperial
9,Av. Presidente Juscelino Kubitschek,Residencial Flamboyant


### Dataframe com os endereços atomizados

In [246]:
enderecos_preprocessed = pd.DataFrame({"logradouro": logradouros, "numero": numeros, "bairro": bairros, "cep": ceps, "cidade": cidades})

enderecos_preprocessed

,logradouro,numero,bairro,cep,cidade
0,Rua 21,Área Especial 4,Jardim Querência,72910-733,ÁGUAS LINDAS DE GOIÁS
1,Avenida Pedro Ludovico,S/N,Reny Cury,75131-457,ANÁPOLIS
2,Avenida Universitária Vereador Vagner da Silva...,Qd. 1 Lt. 1-A,Parque Itatiaia,74968-755,APARECIDA DE GOIÂNIA
3,Rua 02,Qd. 10 Lt. 01 a 15,Centro,76600-000,CIDADE DE GOIÁS
4,Rua 64 esquina com Rua 11,S/N,Expansão Parque Lago,73813-816,FORMOSA
5,Rua 75,46,Centro,74055-110,GOIÂNIA
6,Rua FP. 31,S/N,Recreio dos Funcionários Públicos,74393-290,GOIÂNIA
7,Av. Universitária,S/N,Vale das Goiabeiras,75402-556,INHUMAS
8,Avenida Furnas,55,Bairro Village Imperial,75524-010,ITUMBIARA
9,Av. Presidente Juscelino Kubitschek,775,Residencial Flamboyant,75804-714,JATAÍ


### Dataframe final

In [250]:
df_final = pd.DataFrame(unidades.iloc[:, :2]).join(enderecos_preprocessed).join(unidades.iloc[:, 3:-1])

df_final

,nome,sigla,logradouro,numero,bairro,cep,cidade,site,telefone,email,cnpj,uasg
0,CÂMPUS ÁGUAS LINDAS,LIN,Rua 21,Área Especial 4,Jardim Querência,72910-733,ÁGUAS LINDAS DE GOIÁS,http://www.ifg.edu.br/aguaslindas,(61) 3618-9850,gabinete.aguaslindas@ifg.edu.br,10.870.883/0014-69,158640
1,CÂMPUS ANÁPOLIS,ANA,Avenida Pedro Ludovico,S/N,Reny Cury,75131-457,ANÁPOLIS,http://www.ifg.edu.br/anapolis,(62) 3703-3350,gabinete.anapolis@ifg.edu.br,10.870.883/0009-00,158522
2,CÂMPUS APARECIDA DE GOIÂNIA,APA,Avenida Universitária Vereador Vagner da Silva...,Qd. 1 Lt. 1-A,Parque Itatiaia,74968-755,APARECIDA DE GOIÂNIA,http://www.ifg.edu.br/aparecida,(62) 3507-5950,gabinete.aparecida@ifg.edu.br,10.870.883/0010-35,158610
3,CÂMPUS CIDADE DE GOIÁS,GOI,Rua 02,Qd. 10 Lt. 01 a 15,Centro,76600-000,CIDADE DE GOIÁS,http://www.ifg.edu.br/goias,(62) 3371-9000,gabinete.goias@ifg.edu.br,10.870.883/0011-16,158611
4,CÂMPUS FORMOSA,FOR,Rua 64 esquina com Rua 11,S/N,Expansão Parque Lago,73813-816,FORMOSA,http://www.ifg.edu.br/formosa,(61) 3642 9450,gabinete.formosa@ifg.edu.br,10.870.883/0007-30,158523
5,CÂMPUS GOIÂNIA,GYN,Rua 75,46,Centro,74055-110,GOIÂNIA,http://www.ifg.edu.br/goiania,(62) 3227-2700,dg.goiania@ifg.edu.br,10.870.883/0002-25,158430
6,CÂMPUS GOIÂNIA OESTE,OES,Rua FP. 31,S/N,Recreio dos Funcionários Públicos,74393-290,GOIÂNIA,http://www.ifg.edu.br/goianiaoeste,(62) 3237-1850,gabinete.goianiaoeste@ifg.edu.br,10.870.883/0013-88,158641
7,CÂMPUS INHUMAS,INH,Av. Universitária,S/N,Vale das Goiabeiras,75402-556,INHUMAS,http://www.ifg.edu.br/inhumas,(62) 3514-9501,gabinete.inhumas@ifg.edu.br,10.870.883/0004-97,158432
8,CÂMPUS ITUMBIARA,ITU,Avenida Furnas,55,Bairro Village Imperial,75524-010,ITUMBIARA,http://www.ifg.edu.br/itumbiara,(64) 2103-5600,gabinete.itumbiara@ifg.edu.br,10.870.883/0005-78,158433
9,CÂMPUS JATAÍ,JAT,Av. Presidente Juscelino Kubitschek,775,Residencial Flamboyant,75804-714,JATAÍ,http://www.ifg.edu.br/jatai,(64) 3605-0807,gabinete.jatai@ifg.edu.br,10.870.883/0003-06,158431


In [251]:
export_path = Path("../data/preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    df_final.to_csv(export_path / "unidades.csv", sep=";", index=False)
except Exception as e:
    print(str(e))